In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
import xlrd
import pandas as pd

#original data used to stadarise replicated data
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)

#replicated data
book = xlrd.open_workbook('prepareddatakfold.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7200)]

#create function for creating sequences
def create_inout_sequences(input_data, tw):
    training_seq = []
    label_seq = []
    L = len(input_data)
    for i in range(0,L,tw):
        train_seq = input_data[i:i+tw,0:5]
        train_label = input_data[i:i+tw,5:8]
        training_seq.append(train_seq)
        label_seq.append(train_label)
    return training_seq,label_seq

#standardise data
standardised_data = scaler.transform(replicated_data)
x = torch.tensor(standardised_data)
#make sequences
z, t = create_inout_sequences(x, 12)
training_seq = torch.stack(z)
label_seq = torch.stack(t)
#put into batches
batch_size = 8
train = torch.utils.data.TensorDataset(training_seq,label_seq)
train_loader = torch.utils.data.DataLoader(dataset=train,
                                           batch_size=batch_size, 
                                           shuffle=False)


In [ ]:
#define model
import torch.nn as nn
import torch
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, hn = self.rnn(x, h0.detach())
        out = self.fc(out) 
        return out

In [ ]:
#define architecture
input_dim = 5
hidden_dim = 15
layer_dim = 1
output_dim = 3

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
loss_fn = torch.nn.MSELoss(reduction='mean')

In [ ]:
from tqdm import tqdm
from torch.autograd import Variable
model.train()
learning_rate = 0.002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 50

#train model 
for epoch in range (epochs+1):
    for step, (x, y) in enumerate(train_loader):        # gives batch data
        b_x = Variable(x.view(-1, 12, 5))  
        b_y = Variable(y.view(-1, 12, 3))
    # Forward pass
        optimizer.zero_grad()
        y_pred = model(b_x.float())
    # Compute and print loss.
        loss = loss_fn(y_pred, b_y.float())
        loss.backward()
        optimizer.step()
    print(epoch, loss.item())


#compare predictions with targets
print(b_y)    
print(y_pred)

In [ ]:
#save model
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/RNN1.pt"))                                           